### 從 攝影機 或 影片檔 去辨識人臉

In [2]:
import cv2
import sys
 
def CatchUsbVideo(window_name, camera_idx):
    cv2.namedWindow(window_name)
    
    # 影像來源
    cap = cv2.VideoCapture(camera_idx)                
    
    # keras 提供的人臉辨識器
    classfier = cv2.CascadeClassifier(r"C:\Users\user\PycharmProjects\TensorFlow-Keras\OpenCV\haarcascade_frontalface_default.xml")
    
    # 人臉框的顏色
    color = (0, 255, 0)
        
    while cap.isOpened():
        
        #讀取一幀影像
        ok, frame = cap.read() 
        if not ok:            
            break  
 
        # RGB 轉成 灰階圖減低運算
        grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)                 
        
        # 人臉檢測
        faceRects = classfier.detectMultiScale(grey, scaleFactor = 1.2, minNeighbors = 4, minSize = (32, 32))
        
        # len(faceRects) 表示偵測到的人臉數
        if len(faceRects) > 0:     
            
            # 每張人臉的位置
            for faceRect in faceRects:
                
                # 左上角座標 (x, y)、寬高 (w,h)
                x, y, w, h = faceRect  
                
                # 框出人臉
                cv2.rectangle(frame, (x , y), (x + w , y + h), color, 2)
                        
        # 顯示畫面
        cv2.imshow(window_name, frame)        
        c = cv2.waitKey(10)
        if c & 0xFF == ord('q'):
            break        
    
    # 關閉顯示框 
    cap.release()
    cv2.destroyAllWindows() 
    
CatchUsbVideo("FACE", 0)

### 對影片中的每幀畫面做擷取圖像並儲存

In [ ]:
import cv2
import sys
import matlibplot.pyplot as plt
%matlibplot inline

def CatchUsbVideo(window_name, camera_idx, catch_pic_num, path_name):
    
    cv2.namedWindow(window_name)
    
    cap = cv2.VideoCapture(camera_idx)                
    
    classfier = cv2.CascadeClassifier(r"C:\Users\user\PycharmProjects\TensorFlow-Keras\OpenCV\haarcascade_frontalface_default.xml")
    
    color = (0, 255, 0)
        
    num=0
    
    while cap.isOpened():
        
        ok, frame = cap.read() 
        if not ok:            
            break  

        grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)                 
        
        faceRects = classfier.detectMultiScale(grey, scaleFactor = 1.2, minNeighbors = 3, minSize = (32, 32))

        if len(faceRects) > 0: 
                                   
            for faceRect in faceRects:
                
                x, y, w, h = faceRect 
                
                # 將當前畫面儲存為 JPG 圖片檔
                # %s 是路徑， %d 是號碼
                img_name = '%s/%d.jpg'%(path_name, num)   
                
                # 框出人臉位置存入變數 image
                image = frame[y - 10: y + h + 10, x - 10: x + w + 10]
                
                # 儲存
                cv2.imwrite(img_name, image)                                
                                
                num += 1  
                
                # 超過擷取數量便停止擷取
                if num > (catch_pic_num):   
                    break
                
                # 擷取時畫面的人臉框
                cv2.rectangle(frame, (x - 10, y - 10), (x + w + 10, y + h + 10), color, 2)
                
                # 在框上顯示目前擷取的數量
                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(frame,'num:%d'%(num),(x+30,y+30),font,1,(255,0,225),4)
                
                        
        
        cv2.imshow(window_name, frame)
#         plt.imshow(window_name, frame)    
#         plt.show()    
        c = cv2.waitKey(10)
    
        if c & 0xFF == ord('q'):
            break        
    
   
    cap.release()
    cv2.destroyAllWindows() 
    
    
# 儲存路徑
PATH = r"C:\\Users\\user\\PycharmProjects\\TensorFlow-Keras\\OpenCV\\data\\Mother"

# 將儲存路徑 PATH 代入擷取函數 CatchUsbVideo，擷取 1000 多張
CatchUsbVideo("CUT_face",0,1000,PATH)

### 尺寸調整函數

In [4]:
import cv2
import numpy as np
import sys
import os

IMAGE_SIZE = 224

#按照指定影象大小調整尺寸
def resize_image(image, height = IMAGE_SIZE, width = IMAGE_SIZE):
    
    top, bottom, left, right = (0, 0, 0, 0)
    
    #獲取影象尺寸
    h, w, _ = image.shape
    
    #對於長寬不相等的圖片，找到最長的一邊
    longest_edge = max(h, w)    
    
    #計算短邊需要增加多上畫素寬度使其與長邊等長
    if h < longest_edge:
        dh = longest_edge - h
        top = dh // 2
        bottom = dh - top
    elif w < longest_edge:
        dw = longest_edge - w
        left = dw // 2
        right = dw - left
    else:
        pass 
    
    #RGB顏色
    BLACK = [0, 0, 0]
    
    #給影象增加邊界，是圖片長、寬等長，cv2.BORDER_CONSTANT指定邊界顏色由value指定
    constant = cv2.copyMakeBorder(image, top , bottom, left, right, cv2.BORDER_CONSTANT, value = BLACK)
    
    #調整影象大小並返回
    return cv2.resize(constant, (height, width))

### 用 VGG16 訓練分類模型 ( 在 Colab 訓練 )

In [1]:
from keras.preprocessing.image import img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model


# 載入 keras 內建好的 vgg16 模型
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input


# 標註標籤用
from sklearn.preprocessing import LabelBinarizer
# 劃分訓練集、測試集用
from sklearn.model_selection import train_test_split
# 評分用
from sklearn.metrics import recall_score


import numpy as np
import os


import matplotlib as mlp
import matplotlib.pyplot as plt
from matplotlib.image import imread

#### 載入 VggNet，去除全連接層

In [ ]:
model_vgg = VGG16(weights='imagenet', include_top=False)

#### 圖片處理函數

In [ ]:
def modelProcess(img_path, model):
    
    # 讀取圖片，轉成 224 x 224 大小 
    img = load_img(img_path, target_size=(224, 224)) 

    # 把numpy矩陣中的整數轉換成浮點數，以便於正規化 ./255 的計算
    img = img_to_array(img)

    # 多加一個維度載入 VGG 
    x = np.expand_dims(img, axis=0) 

    # 加載單個圖像時，將獲得一個圖像的形狀，即 (size1,size2,channels)
    # 為了創建一批圖像，您需要一個附加的維度 : (samples, size1,size2, channels)
    # preprocess_input 函數旨在使圖像適合模型所需的格式
    # 正規化
    x = preprocess_input(x) 

    # 特徵提取，全連接層之前的動作
    x_vgg = model.predict(x) 

    # shape（1,7,7,512）
    # 攤平準備掛接全連接層
    x_vgg = x_vgg.reshape(1, 25088) 

    return x_vgg

#### 從資料夾讀取影像資料

In [ ]:
def read_data(path):
    
    folders = os.listdir(path)             # 讀取訓練集內的所有類別資料夾
    for j in range(len(folders)):          # 總共 3 類
        folder = path + '/' + folders[j]   # 各類資料夾的路徑
        dirs = os.listdir(folder)          # 每類的圖片路徑
        
        # 所有圖片路徑列表
        img_path = []
        for i in dirs:
            if os.path.splitext(i)[1] == ".jpg": 
                img_path.append(i)
                
        # 絕對路徑
        img_path = [folder + "/" + i for i in img_path] 
        
        
        # 訓練圖片處理
        # 定義待儲存的 list
        features1 = np.zeros([len(img_path), 25088]) 
        
        for i in range(len(img_path)):
            
            # 對每張圖片進行處理及特徵提取
            feature_i = modelProcess(img_path[i], model_vgg) 
            
            # 印出每張圖的路徑
            print('preprocessed:', img_path[i])
            
            # 圖像資料儲存
            features1[i] = feature_i
        
        # 將所有處理後並攤平的圖像資料放入 X
        if j == 0: 
            X = features1 
        else:
            X = np.concatenate((X, features1), axis=0)
            
    return X 

#### 標籤函數

In [ ]:
def read_label(path):
    y = []
    folders = os.listdir(path)                   # 訓練集路徑
    for j in range(len(folders)): 
        dirName = path + '/' + folders[j] + '/'  # 各類路徑
        lens = len(os.listdir(dirName))          # 每類的圖片數量
        for i in range(lens):
            y.append(j)                          # 有多少張圖，就給多少標籤; 注意，這裡是給 j 
    lb = LabelBinarizer() 
    y = lb.fit_transform(y)                      # ONE_HOT
    return y

#### 上標籤

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/DATA' # 訓練集的資料夾路徑

y = read_label(path)                                 # 上標籤

X = read_data(path)                                  # 依照印出的先後知道各類的標籤碼為何

#### 劃分訓練集、驗證集

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=50)
print('X_train.shape:', X_train.shape)
print('X_test.shape:', X_test.shape)
print('y_train.shape:', y_train.shape)
print('y_test.shape:', y_test.shape)

#### 搭建全連接層

In [ ]:
# 初始化
model = Sequential()

# 輸入維度是 25088
model.add(Dense(units=40, activation='relu', input_dim=25088))

# 我們分三類 ( 2 類以上 )，用 softmax ; 若是二分類，用 sigmoid
model.add(Dense(units=3, activation='softmax'))

# 查看結構
model.summary()

#### 訓練

In [ ]:
# 訓練器是 Adam，二分類用 binary_crossentropy，評估指標用準確率 accuracy
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
 
# 訓練
model.fit(X_train, y_train, epochs=500)

#### 模型儲存

In [ ]:
# model.save('Eric_Chen_3.h5')

### 即時人臉偵測及分類

In [5]:
import cv2
import sys
import gc
import numpy as np
from keras.utils import np_utils
from keras.models import load_model
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input


# 載入vgg模型，去除全連接層
model_vgg = VGG16(weights='imagenet', include_top=False)
    
# 載入全連接層分類模型
model = load_model(r"C:\Users\user\PycharmProjects\TensorFlow-Keras\OpenCV\Eric_Chen_3.h5")    
              
# 框住人臉的矩形邊框顏色       
color = (0, 255, 0)
    
# 抓取攝像機的畫面資料
cap = cv2.VideoCapture(0)
    
# 人臉識別分類器儲存路徑
cascade_path = r"C:\Users\user\PycharmProjects\TensorFlow-Keras\OpenCV\haarcascade_frontalface_default.xml" 
    
# 迴圈檢測識別人臉
while True:
    
        # 每偵畫面
        ret, frame = cap.read()   
        
#         if ret is True:
            
#             #影象灰化，降低計算複雜度
#             frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#         else:
#             continue
            
        #使用人臉識別分類器，讀入分類器
        cascade = cv2.CascadeClassifier(cascade_path)                

        #利用分類器識別出哪個區域為人臉
        faceRects = cascade.detectMultiScale(frame, scaleFactor = 1.2, minNeighbors = 3, minSize = (32, 32))        
        if len(faceRects) > 0:                 
            for faceRect in faceRects: 
                x, y, w, h = faceRect
                
                # 擷取臉部影象提交給模型識別這是誰 
                # 左上角(x、y） 以及 長、寬（h、w)
                image = frame[y - 10: y + h + 10, x - 10: x + w + 10]
        
                # resize
                image = resize_image(image)
                    
#                 #浮點並歸一化
#                 image = image.astype('float32')
#                 image /= 255
                
                image = img_to_array(image)
                xg = np.expand_dims(image, axis=0)
                xg = preprocess_input(xg)
                x_vgg = model_vgg.predict(xg)
                x_vgg = x_vgg.reshape(1, 25088)

                
                #給出輸入屬於各個類別的概率，我們是二值類別，則該函式會給出輸入影象屬於 0 和 1 的概率各為多少
                result1 = model.predict_proba(x_vgg)
                
                # 給出類別預測：0 或者 1
                result = model.predict_classes(x_vgg)
            
                # 辨識出的類別
                faceID = result[0]
                
                x1=int(x - 5)
                x2=int(y - 5)
                y1=int(x + w + 5)
                y2=int(y + h + 5)
                
                #如果是 " 我 "
                if faceID == 2:                                                        
                    cv2.rectangle(frame, (x1, x2), (y1, y2), color, thickness = 2)
                    
                    #文字提示是誰
                    cv2.putText(frame,"ME", 
                                (x + 30, y + 30),                      #座標
                                cv2.FONT_HERSHEY_SIMPLEX,              #字型
                                1,                                     #字號
                                (255,0,255),                           #顏色
                                2)                                     #字的線寬
                    
                # 如果是 " 爸爸 "   
                elif faceID == 1:                                                        
                    cv2.rectangle(frame, (x1, x2), (y1, y2), color, thickness = 2)
                    
                    
                    cv2.putText(frame,"Fther", 
                                (x + 30, y + 30),                      
                                cv2.FONT_HERSHEY_SIMPLEX,              
                                1,                                     
                                (255,0,255),                           
                                2)                                     
                
                # 如果是 " 媽媽 "
                else:
                    cv2.rectangle(frame, (x1, x2), (y1, y2), color, thickness = 2)
                    
                    
                    cv2.putText(frame,"Mother", 
                                (x + 30, y + 30),                      
                                cv2.FONT_HERSHEY_SIMPLEX,              
                                1,                                     
                                (255,0,255),                           
                                2)                                     
                    
                            
        cv2.imshow("Family", frame)
        
        k = cv2.waitKey(1)
        
        if k & 0xFF == ord('q'):
            break


cap.release()
cv2.destroyAllWindows()

Instructions for updating:
Please use `model.predict()` instead.
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
